Последовательность выполнения:

1. Выберите на каком-нибудь сайте объявлений или в интернет-магазине интересующую вас рубрику самого нижнего уровня (например, Для дома и дачи – Мебель и интерьер –
Кровати, диваны и кресла – Диваны).
2. Убедитесь в том, что сайт учел в выдаче ваше местоположение (если нет – укажите его).
3. Обратите внимание на адрес полученной вами страницы и на то, как он изменяется при переходе к следующей странице выдачи (где и как указывается номер страницы), зафиксируйте это.
4. Откройте код страницы выдачи (F12 или правой кнопкой мыши).
Выберите ключевые элементы блока записи об одном объявлении так, чтобы вы могли выделить этот блок из всего текста: например, это может быть определенный тег, с определенным набором атрибутов.


## Взял сайт Крым Мебель